# Set Up

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read in Data

In [2]:
train_df = pd.read_csv("https://raw.githubusercontent.com/derekqin8/Caltech-CS155-2022/main/mps/CS155_mp1/data/LOANS_TRAIN.csv")
test_df = pd.read_csv("https://raw.githubusercontent.com/derekqin8/Caltech-CS155-2022/main/mps/CS155_mp1/data/LOANS_TEST.csv")

In [3]:
train_df.head()

,id,loan_amnt,term_(months),int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,loan_status
0,0,5000,36,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,credit_card,Computer,860xx,AZ,27.65,Jan-1985,3,0,13648,83.7%,9,f,Individual,NaN,0.0,Fully Paid
1,1,2500,60,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,car,bike,309xx,GA,1.00,Apr-1999,3,0,1687,9.4%,4,f,Individual,NaN,0.0,Charged Off
2,2,2400,36,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,small_business,real estate business,606xx,IL,8.72,Nov-2001,2,0,2956,98.5%,10,f,Individual,NaN,0.0,Fully Paid
3,3,10000,36,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,other,personel,917xx,CA,20.00,Feb-1996,10,0,5598,21%,37,f,Individual,NaN,0.0,Fully Paid
4,4,3000,60,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,other,Personal,972xx,OR,17.94,Jan-1996,15,0,27783,53.9%,38,f,Individual,NaN,0.0,Fully Paid


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197250 entries, 0 to 197249
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    197250 non-null  int64  
 1   loan_amnt             197250 non-null  int64  
 2   term_(months)         197250 non-null  int64  
 3   int_rate              197250 non-null  object 
 4   installment           197250 non-null  float64
 5   grade                 197250 non-null  object 
 6   sub_grade             197250 non-null  object 
 7   emp_title             184917 non-null  object 
 8   emp_length            189334 non-null  object 
 9   home_ownership        197250 non-null  object 
 10  annual_inc            197250 non-null  float64
 11  verification_status   197250 non-null  object 
 12  issue_d               197250 non-null  object 
 13  purpose               197250 non-null  object 
 14  title                 197232 non-null  object 
 15  

# Preprocess Data

In [5]:
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder

In [6]:
def preprocess(train_df, test_df):
    ndf = pd.concat([train_df, test_df], axis=0)

    ord_enc = OrdinalEncoder()
    lbl_enc = LabelEncoder()
    scaler = StandardScaler()
    # id: drop
    ndf.drop('id', axis=1, inplace=True)
    # loan_amnt:
    ndf['loan_amnt'] = (ndf['loan_amnt'] - ndf['loan_amnt'].mean()) / ndf['loan_amnt'].std()
    # term_(months):
    ndf['36_months'] = (ndf['term_(months)'] == 36)
    ndf.drop('term_(months)', axis=1, inplace=True)
    # int_rate:
    ndf['int_rate'] = ndf['int_rate'].str.rstrip('%').apply(float)
    ndf['int_rate'] = (ndf['int_rate'] - ndf['int_rate'].mean()) / ndf['int_rate'].std()
    # installment: no need for processing
    ndf['installment'] = (ndf['installment'] - ndf['installment'].mean()) / ndf['installment'].std()
    # grade:
    ndf['grade'] = ord_enc.fit_transform(ndf[['grade']]) + 1
    # sub_grade:
    ndf['sub_grade'] = ord_enc.fit_transform(ndf[['sub_grade']]) + 1
    # emp_title: not useful
    ndf.drop('emp_title', axis=1, inplace=True)
    # emp_length:
    # ndf['emp_length_num'] = ndf["emp_length"].str.replace("year","", regex=True)
    # ndf['emp_length_num'] = ndf['emp_length_num'].str.replace('s', '', regex=True)
    # ndf['emp_length_num'] = ndf['emp_length_num'].str.replace('10', '15', regex=True)
    # ndf['emp_length_num'] = ndf['emp_length_num'].str.replace('+', '', regex=True)
    # ndf['emp_length_num'] = ndf['emp_length_num'].str.replace('< 1', '0', regex=True)
    # ndf['emp_length_num'] = ndf['emp_length_num'].fillna('-1')

    # ndf['emp_length_num'] = ndf['emp_length_num'].apply(int)

    ndf.drop('emp_length', axis=1, inplace=True)
    # home_ownership:
    ndf = pd.get_dummies(ndf, prefix='ho', columns=['home_ownership'])
    # annual_inc:
    ndf['annual_inc'] = (ndf['annual_inc'] - ndf['annual_inc'].mean()) / ndf['annual_inc'].std()
    # verification_status:
    ndf = pd.get_dummies(ndf, prefix='var', columns=['verification_status'])
    # issue_d:
    # ndf['issue_yr'] = ndf['issue_d'].str[-4:].apply(int)
    ndf.drop('issue_d', axis=1, inplace=True)
    # purpose:
    ndf = pd.get_dummies(ndf, prefix='purp', columns=['purpose'])
    # title: not useful
    ndf.drop('title', axis=1, inplace=True)
    # zip_code: not useful
    ndf.drop('zip_code', axis=1, inplace=True)
    # addr_state:
    # ndf['addr_state'] = lbl_enc.fit_transform(df[['addr_state']])
    # ndf = pd.get_dummies(ndf, prefix='state', columns=['addr_state'])
    ndf.drop('addr_state', axis=1, inplace=True)

    # dti:
    ndf['dti'] = (ndf['dti'] - ndf['dti'].mean()) / ndf['dti'].std()
    # earliest_cr_line: too hard to process :(
    ndf.drop('earliest_cr_line', axis=1, inplace=True)
    # open_acc:
    # pub_rec:
    # revol_bal:
    ndf['revol_bal'] = (ndf['revol_bal'] - ndf['revol_bal'].mean()) / ndf['revol_bal'].std()
    # revol_util:
    ndf['revol_util'] = ndf['revol_util'].str.rstrip('%').apply(float)
    ndf['revol_util'] = (ndf['revol_util'] - ndf['revol_util'].mean()) / ndf['revol_util'].std()
    # tot_acc:
    # initial_list_status:
    ndf['initial_list_status'] = (ndf['initial_list_status'] == 'f')
    # application_type: not useful
    ndf.drop('application_type', axis=1, inplace=True)
    # mort_acc:
    ndf['mort_acc'] = ndf['mort_acc'].fillna(0)
    # pub_rec_bankruptcies:

    X_train = ndf[:train_df.shape[0]].copy()
    X_train['loan_status'] = (X_train['loan_status'] == 'Charged Off')
    X_train[X_train.isnull()] = 0.0

    X_test = ndf[-test_df.shape[0]:].copy()
    X_test.drop('loan_status', axis=1, inplace=True)
    X_test[X_test.isnull()] = 0

    return X_train, X_test

In [7]:
train, test = preprocess(train_df, test_df)

In [8]:
train

,loan_amnt,int_rate,installment,grade,sub_grade,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mort_acc,pub_rec_bankruptcies,loan_status,36_months,ho_ANY,ho_MORTGAGE,ho_NONE,ho_OTHER,ho_OWN,ho_RENT,var_Not Verified,var_Source Verified,var_Verified,purp_car,purp_credit_card,purp_debt_consolidation,purp_educational,purp_home_improvement,purp_house,purp_major_purchase,purp_medical,purp_moving,purp_other,purp_renewable_energy,purp_small_business,purp_vacation,purp_wedding
0,-1.082489,-0.721192,-1.070480,2.0,7.0,-0.809853,1.173827,3.0,0.0,-0.111758,1.125331,9.0,True,0.0,0.0,False,True,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.388118,0.312263,-1.492678,3.0,14.0,-0.708519,-1.654911,3.0,0.0,-0.745461,-1.909420,4.0,True,0.0,0.0,True,False,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.400343,0.466610,-1.392291,3.0,15.0,-1.008264,-0.835479,2.0,0.0,-0.678229,1.729830,10.0,True,0.0,0.0,False,True,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,-0.471231,-0.085908,-0.347531,3.0,11.0,-0.384251,0.361825,10.0,0.0,-0.538253,-1.435623,37.0,True,0.0,0.0,False,True,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,-1.326992,-0.264861,-1.460062,2.0,10.0,0.135928,0.143168,15.0,0.0,0.637125,-0.091837,38.0,True,0.0,0.0,False,False,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197245,-1.510369,-1.336343,-1.545494,1.0,4.0,-0.505852,0.173950,5.0,0.0,-0.644268,-1.088445,10.0,True,0.0,0.0,False,True,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
197246,0.026944,-1.112652,0.093433,1.0,5.0,-0.607185,0.595342,15.0,0.0,0.090893,-0.369580,24.0,True,0.0,0.0,False,True,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
197247,-0.348980,0.048307,-0.195394,2.0,10.0,-0.455185,-0.815312,18.0,0.0,-0.360344,-0.508452,20.0,True,0.0,0.0,False,True,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
197248,2.144952,2.030213,1.885439,6.0,26.0,0.220373,0.867070,12.0,0.0,0.391665,0.868010,23.0,True,4.0,0.0,True,False,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [9]:
test

,loan_amnt,int_rate,installment,grade,sub_grade,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mort_acc,pub_rec_bankruptcies,36_months,ho_ANY,ho_MORTGAGE,ho_NONE,ho_OTHER,ho_OWN,ho_RENT,var_Not Verified,var_Source Verified,var_Verified,purp_car,purp_credit_card,purp_debt_consolidation,purp_educational,purp_home_improvement,purp_house,purp_major_purchase,purp_medical,purp_moving,purp_other,purp_renewable_energy,purp_small_business,purp_vacation,purp_wedding
0,-1.085545,-1.754646,-1.117395,1.0,1.0,-0.083628,-1.618822,7.0,0.0,-0.777885,-1.946180,29.0,True,0.0,0.0,True,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,-1.082489,0.936362,-0.996521,4.0,17.0,-0.032961,1.825551,18.0,0.0,-0.277217,-1.121120,37.0,True,0.0,0.0,True,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.471231,-1.754646,-0.490736,1.0,1.0,-0.541487,1.086788,14.0,0.0,-0.225455,-1.235485,18.0,True,0.0,0.0,True,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.082489,-0.721192,-1.070480,2.0,7.0,-0.505852,-0.038338,8.0,0.0,0.143609,1.043641,10.0,True,0.0,0.0,True,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.017775,-0.611583,0.144036,2.0,7.0,-0.590296,-0.605147,7.0,0.0,-0.204263,-0.214371,13.0,True,0.0,0.0,True,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42475,2.585057,2.104031,2.328084,6.0,27.0,0.647663,0.311937,12.0,0.0,1.786490,1.529692,18.0,True,1.0,0.0,False,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
42476,0.949943,0.101993,0.339114,3.0,11.0,-0.260996,0.256743,7.0,0.0,-0.013903,0.096048,18.0,True,0.0,0.0,False,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
42477,1.973800,1.810995,1.655083,5.0,25.0,0.264284,-0.502187,9.0,0.0,-0.156474,0.745476,11.0,True,0.0,0.0,False,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
42478,0.118632,-0.392365,0.283267,2.0,8.0,-0.708519,0.442495,7.0,0.0,0.095661,1.059979,11.0,True,0.0,0.0,True,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197250 entries, 0 to 197249
Data columns (total 40 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   loan_amnt                197250 non-null  float64
 1   int_rate                 197250 non-null  float64
 2   installment              197250 non-null  float64
 3   grade                    197250 non-null  float64
 4   sub_grade                197250 non-null  float64
 5   annual_inc               197250 non-null  float64
 6   dti                      197250 non-null  float64
 7   open_acc                 197250 non-null  float64
 8   pub_rec                  197250 non-null  float64
 9   revol_bal                197250 non-null  float64
 10  revol_util               197250 non-null  float64
 11  total_acc                197250 non-null  float64
 12  initial_list_status      197250 non-null  bool   
 13  mort_acc                 197250 non-null  float64
 14  pub_

# Model Fit

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score

In [ ]:
n_estimators = 2000
clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=10, class_weight={1: 5, 0: 1})
X = train.drop('loan_status', axis=1)
y = train['loan_status']

clf.fit(X, y)

RandomForestClassifier(class_weight={0: 1, 1: 5}, max_depth=10,
                       n_estimators=2000)

In [ ]:
scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print(scores)
print(sum(scores)/len(scores))

[0.66161541 0.69371974 0.68329965 0.69305196 0.67622726]
0.6815828038790883


In [ ]:
train_probs = clf.predict_proba(X)[:,1]
print(roc_auc_score(y, train_probs))

0.7302820372057093


In [ ]:
probs = clf.predict_proba(test)
charged_off_probs = probs[:, 1]

In [ ]:
print(probs)
print(probs.shape)

[[0.90194689 0.09805311]
 [0.5073532  0.4926468 ]
 [0.78503049 0.21496951]
 ...
 [0.31889785 0.68110215]
 [0.55631432 0.44368568]
 [0.4970669  0.5029331 ]]
(42480, 2)


In [ ]:
submission = pd.DataFrame({'Id': test_df['id'], 'loan_status': charged_off_probs})
# you could use any filename. We choose submission here
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import roc_auc_score

In [ ]:
n_estimators = 500
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5), n_estimators=n_estimators)
X = train.drop('loan_status', axis=1)
y = train['loan_status']

clf.fit(X, y)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=500)

In [ ]:
scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
print(scores)
print(sum(scores)/len(scores))

[0.55542542 0.56446445 0.55607946 0.60086966 0.58286651]
0.5719411008216941


In [ ]:
train_probs = clf.predict_proba(X)[:,1]
print(roc_auc_score(y, train_probs))

0.9151130241943738


In [ ]:
probs = clf.predict_proba(test)
charged_off_probs = probs[:, 1]

In [ ]:
submission = pd.DataFrame({'Id': test_df['id'], 'loan_status': charged_off_probs})
# you could use any filename. We choose submission here
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

,Id,loan_status
0,200000,0.476226
1,200001,0.498674
2,200002,0.496310
3,200003,0.491780
4,200004,0.497838


## SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score

In [ ]:
n_estimators=1000
clf = make_pipeline(StandardScaler(), BaggingClassifier(SVC(kernel='linear', max_iter=100), n_estimators=n_estimators, max_samples=1.0/n_estimators, max_features=5))
X = train.drop('loan_status', axis=1)
y = train['loan_status']

clf.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=100).

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('baggingclassifier',
                 BaggingClassifier(base_estimator=SVC(kernel='linear',
                                                      max_iter=100),
                                   max_features=5, max_samples=0.001,
                                   n_estimators=1000))])

In [ ]:
# scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')
# print(scores)
# print(sum(scores)/len(scores))

In [ ]:
train_probs = clf.predict_proba(X)[:,1]
print(roc_auc_score(y, train_probs))

probs = clf.predict_proba(test)
charged_off_probs = probs[:, 1]

submission = pd.DataFrame({'Id': test_df['id'], 'loan_status': charged_off_probs})
# you could use any filename. We choose submission here
submission.to_csv('submission.csv', index=False)

0.5206800569945786


In [ ]:
submission.head()

,Id,loan_status
0,200000,0.000
1,200001,0.004
2,200002,0.000
3,200003,0.004
4,200004,0.000


## Neural Network

In [33]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score

In [34]:
X = train.drop('loan_status', axis=1)
y = train['loan_status']

In [35]:
batch_size=1024
X_np = torch.from_numpy(np.vstack(X.values).astype(float)).float()
y_np = torch.from_numpy(y.values.astype(float)).float().reshape((y.values.shape[0], 1))

test_np = torch.from_numpy(np.vstack(test.values).astype(float)).float()

print(X_np.shape)
print(y_np.shape)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_np, y_np), batch_size=batch_size, shuffle=True)

torch.Size([197250, 39])
torch.Size([197250, 1])


In [46]:
from torch.nn.modules.activation import Softmax
model = nn.Sequential(
    nn.Linear(X_np.shape[1], 400),
    nn.LeakyReLU(),
    nn.Linear(400, 500),
    nn.LeakyReLU(),
    nn.Linear(500, 1000),
    nn.LeakyReLU(),
    nn.Linear(1000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 1000),
    nn.LeakyReLU(),
    nn.Linear(1000, 1000),
    nn.LeakyReLU(),
    nn.Linear(1000, 600),
    nn.LeakyReLU(),
    nn.Linear(600, 400),
    nn.LeakyReLU(),
    nn.Linear(400, 200),
    nn.LeakyReLU(),
    nn.Linear(200, 100),
    nn.LeakyReLU(),
    nn.Linear(100, 50),
    nn.LeakyReLU(),
    nn.Linear(50, 30),
    nn.LeakyReLU(),
    nn.Linear(30, 1),
    nn.Sigmoid()
)
model = model.cuda()

In [47]:
optimizer=torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.BCELoss()

In [48]:
# Some layers, such as Dropout, behave differently during training
model.train()

for epoch in range(40):
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss
        loss = loss_fn(output, target)

        epoch_loss += loss.item()

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch:', epoch + 1, 'Loss:', epoch_loss / X_np.shape[0])

Train Epoch: 1 Loss: 0.0004622267688635184
Train Epoch: 2 Loss: 0.0003961157609905731
Train Epoch: 3 Loss: 0.0003936787059551863
Train Epoch: 4 Loss: 0.00039388691321374196
Train Epoch: 5 Loss: 0.0003928291365523151
Train Epoch: 6 Loss: 0.00039289135095888823
Train Epoch: 7 Loss: 0.00039411116190283017
Train Epoch: 8 Loss: 0.00039248488067220825
Train Epoch: 9 Loss: 0.00039119677730990607
Train Epoch: 10 Loss: 0.0003908620957941459
Train Epoch: 11 Loss: 0.0003910024631461565
Train Epoch: 12 Loss: 0.0003901336628221287
Train Epoch: 13 Loss: 0.0003905294331307647
Train Epoch: 14 Loss: 0.00039012151964112684
Train Epoch: 15 Loss: 0.00038988941990981686
Train Epoch: 16 Loss: 0.00039029096786513527
Train Epoch: 17 Loss: 0.0003897396031441344
Train Epoch: 18 Loss: 0.00039082720101408483
Train Epoch: 19 Loss: 0.0003897018900993357
Train Epoch: 20 Loss: 0.00038914068585264064
Train Epoch: 21 Loss: 0.0003895470618597304
Train Epoch: 22 Loss: 0.00038978434847939303
Train Epoch: 23 Loss: 0.000388

In [41]:
model.eval()

with torch.no_grad():
    train_probs = model(X_np.cuda())
    print(roc_auc_score(y_np, train_probs.cpu()))
    # train_probs = model(X_np)
    # print(roc_auc_score(y_np, train_probs))

0.69054672285499


In [42]:
probs = model(test_np.cuda()).detach().cpu().numpy().reshape(-1)

In [43]:
print(probs)
print(probs.shape)

[0.01322482 0.16232608 0.04734707 ... 0.36488593 0.12494373 0.19149539]
(42480,)


In [44]:
submission = pd.DataFrame({'Id': test_df['id'], 'loan_status': probs})
# you could use any filename. We choose submission here
submission.to_csv('submission.csv', index=False)

In [45]:
submission.head()

,Id,loan_status
0,200000,0.013225
1,200001,0.162326
2,200002,0.047347
3,200003,0.086487
4,200004,0.094939
